# Event-driven backtesting

Source: https://www.fmz.com/bbs-topic/3600

The vectorised nature of pandas ensures that certain operations on large datasets are extremely rapid. However the forms of vectorised backtester suffer from some drawbacks in the way that trade execution is simulated.

In event-driven software the entire set of calculations is run within an "infinite" loop known as the event-loop or game-loop. The code is continually checking for new events and then performing actions based on these events. In particular it allows the illusion of real-time response handling because the code is continually being looped and events checked for.

Event-driven systems (EDS):
 - Advantages over simpler vectorised systems: 
     - Code Reuse (can be used for both historical backtesting and live trading with minimal switch-out of components). This is not true of vectorised backtesters where all data must be available at once to carry out statistical analysis.
     - Lookahead Bias - With an event-driven backtester there is no lookahead bias as market data receipt is treated as an "event" that must be acted upon. 
     - Realism - Event-driven backtesters allow significant customisation over how orders are executed and transaction costs are incurred. It is straightforward to handle basic market and limit orders, as well as market-on-open (MOO) and market-on-close (MOC), since a custom exchange handler can be constructed.
 - Disadvantages over simpler vectorised systems:
    - more complex to implement and test. There are more "moving parts" leading to a greater chance of introducing bugs. To mitigate this proper software testing methodology such as test-driven development can be employed.
    - slower to execute compared to a vectorised system. 

All below components are wrapped in an event-loop. Components (or objects) EDS:
    - Event: contains a type (such as "MARKET", "SIGNAL", "ORDER" or "FILL") that determines how it will be handled within the event-loop.
    - Event Queue: in-memory Python Queue object that stores all of the Event sub-class objects that are generated by the rest of the software.
    - DataHandler: abstract base class (ABC) that presents an interface for handling both historical or live market data => Strategy and Portfolio modules can thus be reused between both approaches. The DataHandler generates a new MarketEvent upon every heartbeat of the system (see below).
    - Strategy: this ABC takes market data and generates SignalEvents, which are ultimately utilised by the Portfolio object. A SignalEvent contains a ticker symbol, a direction (LONG or SHORT) and a timestamp.
    - Portfolio: this ABC handles the order mgmt associated with current and subsequent positions for a strategy. It also carries out risk management across the portfolio, including sector exposure and position sizing. In a more sophisticated implementation this could be delegated to a RiskManagement class. The Portfolio takes SignalEvents from the Queue and generates OrderEvents that get added to the Queue.
    - ExecutionHandler: simulates a connection to a brokerage. Takes OrderEvents from the Queue and execute them, either via a simulated approach or an actual connection to a liver brokerage. Once orders are executed the handler creates FillEvents, which describe what was actually transacted, including fees, commission and slippage (if modelled).

In [ ]:
# Declare the components with respective parameters
bars = DataHandler(..)
strategy = Strategy(..)
port = Portfolio(..)
broker = ExecutionHandler(..)

while True: # heartbeat loop
    # Update the bars (specific backtest code, as opposed to live trading)
    if bars.continue_backtest == True:
        bars.update_bars()
    else:
        break
    
    # Handle the events
    while True:
        try:
            event = events.get(False) # capture of events from an in-memory queue
        except Queue.Empty: # When the events Queue is empty, the heartbeat loop continues
            break
        else:
            if event is not None:
                if event.type == 'MARKET':
                    strategy.calculate_signals(event)
                    port.update_timeindex(event)

                elif event.type == 'SIGNAL':
                    port.update_signal(event)

                elif event.type == 'ORDER':
                    broker.execute_order(event)

                elif event.type == 'FILL':
                    port.update_fill(event)

    # 10-Minute heartbeat
    # For live trading this is the frequency at which new market data is polled
    # 
    time.sleep(10*60) 

## Events types:


In [ ]:
class Event(object):
    pass

class MarketEvent(Event): 
    # triggered when the outer while loop begins a new "heartbeat"
    # occurs when the DataHandler object receives a new update of market data ...
    # ... for any symbols which are currently being tracked
    #  trigger the Strategy object generating new trading signals
    
    def __init__(self):
        self.type = 'MARKET' # simply contains an identification that it is a market event
        
class SignalEvent(Event): # market data -> Strategy obj -> Signal -> Portfolio obj    
    def __init__(self, symbol, datetime, signal_type):        
        self.type = 'SIGNAL'
        self.symbol = symbol
        self.datetime = datetime # when the signal was generated
        self.signal_type = signal_type # 'LONG' or 'SHORT'
        
class OrderEvent(Event): # -> Portfolio obj -> Order -> Execution obj 
    def __init__(self, symbol, order_type, quantity, direction):   
        self.type = 'ORDER'
        self.symbol = symbol
        self.order_type = order_type # MKT or LMT
        self.quantity = quantity # determined by Portfolio constraints (risk and position sizing)
        self.direction = direction # 'BUY' or 'SELL'

    def print_order(self):
        print "Order: Symbol=%s, Type=%s, Quantity=%s, Direction=%s" % \
            (self.symbol, self.order_type, self.quantity, self.direction)
        
class FillEvent(Event): # Once ExecutionHandler transacted the order, it generates a FillEvent
    def __init__(self, timeindex, symbol, exchange, quantity, 
                 direction, fill_cost, commission=None):
        self.type = 'FILL'
        self.timeindex = timeindex # when the order was filled
        self.symbol = symbol
        self.exchange = exchange # where the order was filled
        self.quantity = quantity
        self.direction = direction # 'BUY' or 'SELL'
        self.fill_cost = fill_cost # holdings value in dollars

        # Calculate commission
        if commission is None:
            self.commission = self.calculate_ib_commission()
        else:
            self.commission = commission

    def calculate_ib_commission(self):
        # https://www.interactivebrokers.com/en/index.php?f=commission&p=stocks2
        
        full_cost = 1.3
        if self.quantity <= 500:
            full_cost = max(1.3, 0.013 * self.quantity)
        else: # Greater than 500
            full_cost = max(1.3, 0.008 * self.quantity)
        full_cost = min(full_cost, 0.5 / 100.0 * self.quantity * self.fill_cost)
        return full_cost